In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import numpy as np 
from matplotlib import pyplot as plt

In [3]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets import IrisDataset
# for QASVM
from classifiers.quantum.qasvm import SoftQASVM, PseudoSoftQASVM
from classifiers.quantum.ansatz import Circuit9, sample_circuit
from classifiers.optimizer import tSPSA
from classifiers.callback import CostParamStorage
from qiskit.utils import QuantumInstance
from qiskit.providers.aer import AerSimulator, StatevectorSimulator, Aer, QasmSimulator

In [21]:
ds = IrisDataset(feature_range=(-np.pi, np.pi), true_hot=2)
X, y = ds.sample(2**6, return_X_y=True)

In [45]:
ds.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
backend = StatevectorSimulator(device='CPU', shots=2**13)
backend.options

Options(shots=8192, device='CPU', precision='double', executor=None, max_job_size=None, max_shot_size=None, zero_threshold=1e-10, validation_threshold=None, max_parallel_threads=None, max_parallel_experiments=None, max_parallel_shots=None, max_memory_mb=None, seed_simulator=None, fusion_enable=True, fusion_verbose=False, fusion_max_qubit=5, fusion_threshold=14, statevector_parallel_threshold=14)

In [24]:
_feature_map = Circuit9(X.shape[1], reps=1)
feature_map = _feature_map.copy()
for _ in range(2 - 1):
    feature_map.compose(_feature_map, inplace=True)

var_form = sample_circuit(15)(num_qubits=6, reps=5)
quantum_instance = QuantumInstance(backend=backend, shots=2**13)
qasvm = SoftQASVM(
        X, y, lamda=1, C=10,
        quantum_instance=quantum_instance,
        var_form=var_form,
        feature_map=feature_map
    )
storage = CostParamStorage()
optimizer = tSPSA(maxiter=2**5, blocking=True, last_avg=16, callback=storage)

In [ ]:
qasvm.cost_fn(qasvm.parameters)

In [ ]:
import qiskit.tools.jupyter
from qiskit.algorithms.optimizers import SPSA, ADAM
from tqdm.notebook import tqdm
MAXITER = 2**10

pbar = tqdm(total=MAXITER)

def callback(nfev, params, fval, stepsize, accepted):
    storage(nfev, parameters=params, cost=fval, step_size=stepsize, isaccepted=accepted)
    pbar.update()

optimizer = SPSA(maxiter=MAXITER, blocking=True, last_avg=16, callback=callback)
# optimizer = ADAM(snapshot_dir='./')

opt_result = optimizer.minimize(fun=qasvm.cost_fn, x0=qasvm.initial_point, jac=qasvm.grad_fn)

In [ ]:
random_params = np.random.random(qasvm.num_parameters)
qasvm.cost_fn(random_params)

In [ ]:
from qiskit import transpile
qc = qasvm.var_form.assign_parameters(dict(zip(qasvm.var_form.parameters, random_params)))
alpha = np.abs(StatevectorSimulator().run(transpile(qc, backend=StatevectorSimulator())).result().get_statevector())**2

In [ ]:
pqasvm.cost_fn(alpha)

In [ ]:
qasvm._evaluate_regularizer_circuit(random_params)

In [ ]:
sum(alpha**2)

In [ ]:
print(opt_result)

In [ ]:
storage.plot()

In [ ]:
qasvm.parameters = storage.last_avg(16)
qasvm.accuracy(X, y)

In [25]:
pqasvm = PseudoSoftQASVM(X, y, quantum_instance=quantum_instance, feature_map=feature_map, lamda=1, C=qasvm.C)
pstorage = CostParamStorage()

In [26]:
pqasvm.cost_fn(pqasvm.parameters)

0.12060156681437949

In [27]:
pqasvm.initial_point = np.where(y>0, 1-sum(y)/len(y), sum(y)/len(y))
pqasvm.parameters = pqasvm.initial_point

In [28]:
from scipy.optimize import minimize

scipy_result = minimize(pqasvm._w_fn, x0=pqasvm.initial_point, method='SLSQP', constraints=dict(type='eq', fun=pqasvm.constraint))
scipy_result

     fun: 0.0244360286289753
     jac: array([-5.20971837e-04,  2.22556293e-04,  3.91353387e-04,  2.13901745e-04,
        4.21479810e-04, -4.31355787e-04, -4.82431846e-04, -4.01244499e-04,
       -7.00720120e-04, -6.08833507e-05, -8.71662050e-05,  2.54473649e-04,
       -2.31562648e-04,  6.04685396e-04,  7.18232943e-04, -3.41805164e-04,
        6.51461538e-04,  1.49717089e-04, -3.77359567e-04, -2.42688018e-03,
       -3.19448067e-04,  2.97821593e-04, -2.71503348e-04, -3.49923037e-04,
        1.84424687e-04,  2.51554884e-05,  4.91858693e-04,  2.33746134e-04,
        6.55625947e-04,  3.95481940e-04, -1.00207515e-04,  3.57290031e-04,
       -3.94925009e-04,  3.97631899e-04,  3.72686423e-04, -2.53985403e-04,
        1.84190180e-03,  1.27383973e-04,  3.80649231e-04,  4.42641089e-04,
       -2.41357833e-04,  2.91964738e-04, -1.69040635e-04, -1.73826702e-04,
       -2.97777820e-04,  4.81766183e-04,  4.63223085e-04,  1.85887795e-04,
       -4.42458317e-04, -1.95937930e-04,  5.51666599e-04,  3.

In [29]:
pqasvm.parameters = scipy_result.x
pqasvm.accuracy(ds.data, ds.target)

0.9333333333333333

In [43]:
from sklearn.svm import SVC

svm = SVC(C=10, kernel='precomputed')
svm.fit(pqasvm.kernel_matrix, y)
testK = np.abs(pqasvm._qk.evaluate(ds.data, X))**2
sum(svm.predict(testK)==ds.target)/len(ds.target)

0.9266666666666666

In [31]:
import qiskit.tools.jupyter
from qiskit.algorithms.optimizers import SPSA, ADAM
from tqdm.notebook import tqdm
MAXITER = 2**10

pbar = tqdm(total=10*MAXITER)

def pcallback(nfev, params, fval, stepsize, accepted):
    pstorage(nfev, parameters=params, cost=fval, step_size=stepsize, isaccepted=accepted)
    pbar.update()

poptimizer = SPSA(maxiter=10*MAXITER, blocking=True, last_avg=16, callback=pcallback)
#poptimizer = ADAM()

popt_result = poptimizer.minimize(fun=pqasvm.cost_fn, x0=pqasvm.initial_point)
pbar.close()

  0%|          | 0/10240 [00:00<?, ?it/s]

In [32]:
print(popt_result)

{   'fun': 0.032293830980844845,
    'jac': None,
    'nfev': 30721,
    'nit': None,
    'njev': None,
    'x': array([-2.03507274e-01,  2.02900998e+00,  6.53796335e-01,  1.52527365e+00,
        6.05677424e-01,  4.77160576e-01, -2.83763944e-01,  2.55481503e-01,
        6.62226589e-01, -1.32872252e+00,  4.46198925e-01,  2.96176239e-01,
        2.73489257e-01,  1.57889742e-01,  9.53231550e-04, -1.42732353e-01,
        2.42607531e+00,  1.53573738e-01,  6.85965834e-02, -6.57869285e-04,
        3.48635329e-01, -1.99631617e-01,  7.73768189e-01, -1.61021198e-02,
        2.54569873e+00,  1.09885391e-02,  4.81987774e-01,  3.25745800e-01,
        2.79772553e-01, -1.06556466e-02,  1.89462629e-02, -1.10594523e-04,
        2.51455557e+00,  9.28420786e-01,  4.10109333e-03,  1.40428964e-01,
        3.10104307e-02, -1.68344793e-02, -1.02933385e+00, -3.34604691e-01,
        3.21575561e-02,  1.78209819e-01,  4.37992294e-01, -2.03667449e+00,
        8.62047480e-01,  5.27038583e-01,  8.66941639e-01,  4.4

In [33]:
#pqasvm.parameters = pstorage.last_avg(16, True)
pqasvm.parameters = popt_result.x
pqasvm.accuracy(ds.data, ds.target)

0.94

In [ ]:
pstorage.plot()

In [ ]:
pqasvm

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table